In [1]:
import numpy as np
import pandas as pd
import scipy as sp

import matplotlib.pyplot as plt

import os
import glob

from nfl import Team, Game, Play, Player

In [2]:
games = pd.read_csv('data/games.csv')

In [3]:
players = pd.read_csv('data/players.csv').set_index('nflId')

In [4]:
plays = pd.read_csv('data/plays.csv')

In [5]:
weeks = {}
for i in range(3):
    name = f'week{i+1}'
    print(name)
    weeks[name] = pd.read_csv(os.path.join('data',name+'-processed.csv'))

week1
week2
week3


In [6]:
team_names = games['homeTeamAbbr'].unique()

In [7]:
teams = {}
for name in team_names:
    teams[name] = Team(abbr=name)

In [8]:
for i in range(games.shape[0]):
    _game_info = games.loc[i]
    gameId = _game_info['gameId']
    homeTeamAbbr = _game_info['homeTeamAbbr']
    visitorTeamAbbr = _game_info['visitorTeamAbbr']
    week = _game_info['week']
    
    game_plays = plays[plays['gameId'] == gameId]
    home_game_plays = game_plays[game_plays['possessionTeam']==homeTeamAbbr].sort_values(by=['playId']).reset_index(drop=True)
    away_game_plays = game_plays[game_plays['possessionTeam']==visitorTeamAbbr].sort_values(by=['playId']).reset_index(drop=True)
    
    teams[homeTeamAbbr].games[f'week{week}'] = Game(gameId,opponent=visitorTeamAbbr,game_info=_game_info,
                                                    play_data=away_game_plays,location='home')
    teams[visitorTeamAbbr].games[f'week{week}'] = Game(gameId,opponent=homeTeamAbbr,game_info=_game_info,
                                                    play_data=home_game_plays,location='away')

In [9]:
team = teams['IND']

In [10]:
for key in weeks.keys():
    game = team.games[key]
    
    week_data = weeks[key]
    
    for i in game.play_data.index:
        play = game.play_data.loc[i]
        tracking_data = week_data[(week_data['gameId']==play['gameId'])&(week_data['playId']==play['playId'])]
        player_tracking = tracking_data[tracking_data['nflId'].notna()]
        fb_tracking = tracking_data[tracking_data['displayName']=='Football'].sort_values(by='frameId').reset_index(drop=True)
        
        game.plays.append(Play(play['playId'],play_data=play,player_tracking=player_tracking,
                               fb_tracking=fb_tracking,defensive_team=game.location))

In [11]:
team.games

{'week1': <nfl.Game at 0x1345f3ac8>,
 'week2': <nfl.Game at 0x10777e630>,
 'week3': <nfl.Game at 0x107795f28>,
 'week4': <nfl.Game at 0x124e1ada0>,
 'week5': <nfl.Game at 0x124e30cf8>,
 'week6': <nfl.Game at 0x124e58a58>,
 'week7': <nfl.Game at 0x124e6b320>,
 'week8': <nfl.Game at 0x124e86a58>,
 'week10': <nfl.Game at 0x125637860>,
 'week11': <nfl.Game at 0x125649da0>,
 'week12': <nfl.Game at 0x125665320>,
 'week13': <nfl.Game at 0x125680518>,
 'week14': <nfl.Game at 0x1256917b8>,
 'week15': <nfl.Game at 0x1256ae5c0>,
 'week16': <nfl.Game at 0x1256b5b00>,
 'week17': <nfl.Game at 0x1256d9a58>}

In [12]:
for _week in weeks.keys():
    game = team.games[_week]
    print(f'{game} - {game.nPlays} Plays')
    
    for i in range(game.nPlays):
        print(f'    Play {i+1}')
        try:
            play = game.plays[i]
            play.process_players(players)
            play.find_dropback_events()
            dst = os.path.join('visualizations',team.abbr,game.info,f'Play {i+1}','images')
            play.plot_play_frames(markers='position',target_directory=dst)
        except:
            print(f'      Play Error!')

Week 1 - IND vs CIN (09-09-2018) - 32 Plays
    Play 1
    Play 2
    Play 3
    Play 4
    Play 5
    Play 6
    Play 7
    Play 8
    Play 9
    Play 10
    Play 11
    Play 12
    Play 13
    Play 14
    Play 15
    Play 16
      Play Error!
    Play 17
    Play 18
    Play 19
    Play 20
    Play 21
    Play 22
    Play 23
      Play Error!
    Play 24
    Play 25
    Play 26
    Play 27
    Play 28
    Play 29
    Play 30
    Play 31
    Play 32
Week 2 - IND at WAS (09-16-2018) - 49 Plays
    Play 1
    Play 2
    Play 3
    Play 4
    Play 5
    Play 6
      Play Error!
    Play 7
    Play 8
    Play 9
    Play 10
    Play 11
    Play 12
    Play 13
    Play 14
    Play 15
    Play 16
    Play 17
      Play Error!
    Play 18
    Play 19
    Play 20
    Play 21
    Play 22
    Play 23
    Play 24
    Play 25
    Play 26
    Play 27
    Play 28
    Play 29
    Play 30
      Play Error!
    Play 31
    Play 32
    Play 33
    Play 34
    Play 35
    Play 36
    Play 37
    Play 38


In [13]:
play.events

{'ball_snap': 11,
 'play_action': 24,
 'pass_forward': 48,
 'pass_arrived': 54,
 'pass_outcome_caught': 57,
 'first_contact': 66,
 'tackle': 74,
 'peak_dropback': 25,
 'end_dropback': 37}